# Shade my Run
## Data Exploration

- [LIDAR Data](https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1537)


From [Summary](https://daac.ornl.gov/CMS/guides/CMS_LiDAR_AGB_California.html):

"This dataset provides estimates of aboveground biomass and spatially explicit uncertainty from 53 airborne LiDAR surveys of locations throughout California between 2005 and 2014. Aboveground biomass was estimated by performing individual tree crown detection and applying a customized "remote sensing aware" allometric equation to these individual trees. Aboveground biomass estimates and their uncertainties for each study area are provided in per-tree and gridded format. The canopy height models used for the tree detection and biomass estimation are also provided.
There are 9,504 files in compressed shapefile format (*.shp contained in *.zip) and 212 files in GeoTIFF format (*.tif) included in this dataset."


In [ ]:
!pip install rasterio
!pip install matplotlib
!pip install pycrs
!pip install pyshp
!pip install pyproj
!pip install ipympl

In [ ]:
# import packages
from map_retrieve import mapRetrieve
from glob import glob
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import os
import logging
from pprint import pprint

# create the mapRetrieve object
mr = mapRetrieve()
# turn on logging to get output from methods
logging.basicConfig(level=logging.INFO)


### Shape files

Each tree is represented by a polygon object which a set of coordinates.   One popular package for dealing with shapes is pyshp.  In addition the files are zipped so we used python's ZipFile library to open the folder in memory without wirting all the files to disk. We use pycrs to extract the projection information we need to align the shape file with other maps. 

In [ ]:
f_name = 'data/Taliaferro2013_3992854N_12308761W.zip'
shape, proj4 = mr.load_shape(f_name)

### Get Bounding Box of shape file

We use the pyproj tools to project the shape bounding box from UTM to geo coordinates, and rerrange the bounding box for the GDAL command later.  


In [ ]:
extents = mr.get_bounds(shape, proj4)

### Get a World Imagery map with the extents of the shape file 

We first request a map from the arcgis World Imagery map server. Note this uses call a GDAL program as a sytem process so it is not purely python. 


In [ ]:
dst_file = 'maps/test_clip.tif'
mr.get_map(extents, dst_file)
with rasterio.open(dst_file) as src:
    show(src)

### Warp Map to match projection of LIDAR data

Next we need to transform the map to match the match the LIDAR data. Again we use a GDAL program as a system process. 

In [ ]:
r_dst_file = 'maps/test_clip_r.tif'
mr.warp_map(src_file=dst_file, dst_file=r_dst_file)


### Get bounding boxes of tree polygons and save to JSON

Here we  visualize the bounding box on one of the polygons

In [ ]:
# add the interactive widget
%matplotlib widget

f = plt.figure()
f.canvas.layout.width = '90%'
#f.canvas.layout.height = '100%'
# f.canvas.layout.width = '2in'

for poly in shape.shapeRecords():
    # plot the original polygons
    x1 = [p[0] for p in poly.shape.points]
    y1 = [p[1] for p in poly.shape.points]
    plt.plot(x1,y1)
    # plot the polygon bounding boxes
    minx, miny, maxx, maxy = poly.shape.bbox
    x2 = [minx, minx, maxx, maxx, minx]
    y2 = [miny, maxy, maxy, miny, miny]
    plt.plot(x2,y2)

# plot the reprojected map    
src = rasterio.open(r_dst_file)
show(src.read(), transform=src.transform)

plt.show()

### Finally we save the bounding box data as a json file. 

In [ ]:
filename = f_name.split("/")[-1]
filename = filename.split(".")[0]
print(filename)


In [ ]:
f_name = r_dst_file
json_file = mr.shape_to_json(shape, src.transform, f_name=f_name)
mr.save_json(json_file, dst_file='labels/test.js')

with open('labels/test.js', 'r') as f:
    head = [next(f) for x in range(10)]
pprint(head)



### Putting it all together 

Now we can just feed in the name of a zip file and save a map image and a corresponding json file with the boudnng boxws labeled. 

In [2]:
# get the names of all the zip files in the data folder
#zip_files = glob('data/*.zip')
import os
from map_retrieve import mapRetrieve
from glob import glob

# create the mapRetrieve object
mr = mapRetrieve()

zip_files = glob('/media/zac/Seagate Portable Drive/orders/f06d9ed2c630d7ad6ecfd53ecda4d412/CMS_LiDAR_AGB_California/data/*.zip')
# for each zip file run the save_map method
count = 100
for zf in zip_files:
    count = count -1
    if count > 0:
        mr.save_map(zf)
    else:
        continue


/media/zac/Seagate Portable Drive/orders/f06d9ed2c630d7ad6ecfd53ecda4d412/CMS_LiDAR_AGB_California/data/Bagley2013_4094993N_12203692W.zip
Bagley2013_4094993N_12203692W.zip
Bagley2013_4094993N_12203692W
-projwin -122.03692581231681 40.94992892704141 -122.03663560221024 40.94978496942493 -ot Byte -of GTiff -co COMPRESS=NONE -co BIGTIFF=IF_NEEDED 'http://server.arcgisonline.com/arcgis/rest/services/ESRI_Imagery_World_2D/MapServer?f=json&pretty=true' maps/temp.gif
/media/zac/Seagate Portable Drive/orders/f06d9ed2c630d7ad6ecfd53ecda4d412/CMS_LiDAR_AGB_California/data/Bagley2013_4095007N_12204966W.zip
Bagley2013_4095007N_12204966W.zip
Bagley2013_4095007N_12204966W
-projwin -122.04974149258646 40.94996013536803 -122.0368668313569 40.94468235116513 -ot Byte -of GTiff -co COMPRESS=NONE -co BIGTIFF=IF_NEEDED 'http://server.arcgisonline.com/arcgis/rest/services/ESRI_Imagery_World_2D/MapServer?f=json&pretty=true' maps/temp.gif
/media/zac/Seagate Portable Drive/orders/f06d9ed2c630d7ad6ecfd53ecda4d4

In [ ]:
# cleaning up
try:
    os.remove('labels/test.js')
except:
    pass
try:
    os.remove('maps/test_clip_r.tif')
except:
    pass